<a href="https://colab.research.google.com/github/adc257/info4940-sitcom/blob/main/sitcom_T5_(with_nContext).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Big Bang Theory T5 nContext

# Imports:

In [52]:
# from google.colab import drive
# drive.mount('/content/drive')

In [53]:
!git clone https://github.com/adc257/info4940-sitcom.git

fatal: destination path 'info4940-sitcom' already exists and is not an empty directory.


In [54]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [55]:
!python -m venv 3350
!source 3350/bin/activate

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/local/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/venv/__main__.py", line 6, in <module>
    main()
  File "/usr/local/lib/python3.10/venv/__init__.py", line 516, in main
    builder.create(d)
  File "/usr/local/lib/python3.10/venv/__init__.py", line 75, in create
    self._setup_pip(context)
  File "/usr/local/lib/python3.10/venv/__init__.py", line 328, in _setup_pip
    self._call_new_python(context, '-m', 'ensurepip', '--upgrade',
  File "/usr/local/lib/python3.10/venv/__init__.py", line 324, in _call_new_python
    subprocess.check_output(args, **kwargs)
  File "/usr/local/lib/python3.10/subprocess.py", line 421, in check_output
    return run(*popenargs, stdout=PIPE, timeout=timeout, check=True,
  File "/usr/local/lib/python3.10/subprocess.py", l

In [56]:
# !pip install transformers
# !pip install transformers[torch]
# !pip install accelerate -U
# !pip install accelerate>=0.21.0
# !pip install --upgrade pip

In [57]:
# pip uninstall transformers accelerate
# pip install transformers[torch]
# !pip install --upgrade setuptools

In [58]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, time
from pathlib import Path

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import os
from sklearn.metrics import f1_score, classification_report
from sklearn.ensemble import RandomForestClassifier

In [59]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from transformers import T5Tokenizer, T5ForConditionalGeneration


# Formatting the Dialogue:

In [6]:
def list_files(start_path):
    file_paths = []
    for root, dirs, files in os.walk(start_path):
        for file in files:
            file_paths.append(os.path.join(root, file))

    file_paths.sort()
    return file_paths

In [7]:
def transformContextTarget(context, target):
  separator = "\n"
  quotation_token = '"'

  formatted_input = ""

  current_scene = context[0]['Scene']

  formatted_input += "Use the following as context:" + separator*2
  formatted_input += "The scene is: " + current_scene + separator

  for entry in context:
    if current_scene != entry['Scene']:
      current_scene = entry["Scene"]
      formatted_input += "The scene changes to: " + current_scene + separator

    recipients = entry['Recipients']
    recipients_str = ""
    if len(recipients) == 0:
      recipients_str += "to themselves"
    elif len(recipients) == 1:
      recipients_str += recipients[0]
    else:
      recipients_str += ", ".join(recipients[:-1]) + " and " + recipients[-1]

    formatted_input += f"{entry['Speaker']} says {quotation_token}{entry['Dialogue']}{quotation_token} to {recipients_str}."
    formatted_input += separator


  formatted_input += separator + "-" *20 + separator*2

  formatted_input += "Using the previous dialogues above as context, classify the following dialogue below as humorous or not humorous:" + separator*2


  entry = target
  if entry['Scene'] != current_scene:
    current_scene = entry["Scene"]
    formatted_input += "The scene changes to: "
  else:
    formatted_input += "The Scene is still: "

  formatted_input += current_scene + separator


  recipients = entry['Recipients']
  recipients_str = ""
  if len(recipients) == 0:
    recipients_str += "to themselves"
  elif len(recipients) == 1:
    recipients_str += recipients[0]
  else:
    recipients_str += ", ".join(recipients[:-1]) + " and " + recipients[-1]

  formatted_input += f"{entry['Speaker']} says {quotation_token}{entry['Dialogue']}{quotation_token} to {recipients_str}."
  formatted_input += separator*2 + "Classify: [humorous] or [not humorous]."

  return formatted_input

# context = [data[0], data[1], data[2]]
# target = data[3]
# transformContextTarget(context, target)

In [8]:
filePath = "/content/info4940-sitcom/cleaned-data/S1"
nSentenceContext = 3

episodePath = list_files(filePath)[0]

with open(episodePath, 'r') as file:
  data = json.load(file)
  data = [info for (_, info) in data.items()]

print(f"Example raw entry:")
display(data[3])

print("\n\nExample formatted entry (3context):\n")
print(transformContextTarget(data[:3], data[3]))

print(f'Actual: {("isHumor" in data[3])}')

Example raw entry:


{'EpisodeID': 'The Big Bang_S0101',
 'Scene': 'A corridor at a sperm bank.',
 'Recipients': ['Sheldon', 'Receptionist'],
 'Speaker': 'Leonard',
 'Dialogue': 'Excuse me.',
 'Dialogue Start Time': '00:00:23:140000',
 'Dialogue End Time': '00:00:25:060000'}



Example formatted entry (3context):

Use the following as context:

The scene is: A corridor at a sperm bank.
Sheldon says "So if a photon is directed through a plane with two slits in it and either slit is observed, it will not go through both slits. If it's unobserved, it will. However, if it's observed after it's left the plane but before it hits its target, it won't have gone through both slits." to Leonard.
Leonard says "Agreed. What's your point?" to Sheldon.
Sheldon says "There's no point, I just think it's a good idea for a T-shirt." to Leonard.

--------------------

Using the previous dialogues above as context, classify the following dialogue below as humorous or not humorous:

The Scene is still: A corridor at a sperm bank.
Leonard says "Excuse me." to Sheldon and Receptionist.

Classify: [humorous] or [not humorous].
Actual: False


# Train Test Split:

In [22]:
def generateDialogueWithContext(filePath, nSentenceContext):
  formatted_inputs = []
  labels = []

  for episodePath in list_files(filePath):

    with open(episodePath, 'r') as file:
      data = json.load(file)
      data = [info for (_, info) in data.items()]


      i = nSentenceContext
      while i < len(data):
        context = [data[j] for j in range(i-nSentenceContext, i)]
        target = data[i]

        formatted_inputs.append(transformContextTarget(context, target))

        if "isHumor" in target:
          labels.append(1)
        else:
          labels.append(0)

        i += 1
  return formatted_inputs, labels

In [48]:
dialogue_list1, label_list1 = generateDialogueWithContext('/content/info4940-sitcom/cleaned-data/S1', 3)
dialogue_list2, label_list2 = generateDialogueWithContext('/content/info4940-sitcom/cleaned-data/S2', 3)
dialogue_list3, label_list3 = generateDialogueWithContext('/content/info4940-sitcom/cleaned-data/S3', 3)
dialogue_list4, label_list4 = generateDialogueWithContext('/content/info4940-sitcom/cleaned-data/S4', 3)
dialogue_list5, label_list5 = generateDialogueWithContext('/content/info4940-sitcom/cleaned-data/S5', 3)

In [49]:
train_texts = dialogue_list1 + dialogue_list2 + dialogue_list3 + dialogue_list4
test_texts = dialogue_list5
train_labels = label_list1 + label_list2 + label_list3 + label_list4
test_labels = label_list5

# T5: